<a href="https://colab.research.google.com/github/cesaroliveiragoes/Statistics-for-Data-Science/blob/main/checkpoint_01_2_sem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Checkpoint 1 - Análise Covid Brasil

## Carregar módulos

In [ ]:
import numpy as np
import pandas as pd

## Carregar dados

- Dados Covid Brasil

In [ ]:
! wget --no-check-certificate --content-disposition 'https://github.com/wcota/covid19br/blob/master/cases-brazil-cities-time.csv.gz?raw=true'

--2023-08-31 22:27:23--  https://github.com/wcota/covid19br/blob/master/cases-brazil-cities-time.csv.gz?raw=true
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/wcota/covid19br/raw/master/cases-brazil-cities-time.csv.gz [following]
--2023-08-31 22:27:23--  https://github.com/wcota/covid19br/raw/master/cases-brazil-cities-time.csv.gz
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-cities-time.csv.gz [following]
--2023-08-31 22:27:23--  https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-cities-time.csv.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|18

In [ ]:
! gunzip cases-brazil-cities-time.csv.gz

In [ ]:
dados_brasil = pd.read_csv('cases-brazil-cities-time.csv')

In [ ]:
dados_brasil

,epi_week,date,state,city,ibgeID,newDeaths,deaths,newCases,totalCases,deaths_per_100k_inhabitants,totalCases_per_100k_inhabitants,deaths_by_totalCases
0,301,2023-01-01,GO,Abadia de Goiás/GO,5200050,0,49,0,3952,535.051,43153.527,0.012
1,301,2023-01-01,MG,Abadia dos Dourados/MG,3100104,0,20,0,1929,284.819,27470.806,0.010
2,301,2023-01-01,GO,Abadiânia/GO,5200100,0,55,0,2154,263.498,10319.552,0.026
3,301,2023-01-01,PA,Abaetetuba/PA,1500107,0,243,0,11776,151.459,7339.861,0.021
4,301,2023-01-01,MG,Abaeté/MG,3100203,0,48,0,4130,206.336,17753.514,0.012
...,...,...,...,...,...,...,...,...,...,...,...,...
430964,311,2023-03-18,PR,Ângulo/PR,4101150,0,11,0,967,375.299,32992.153,0.011
430965,311,2023-03-18,BA,Érico Cardoso/BA,2900504,0,53,13,2467,504.138,23466.185,0.021
430966,311,2023-03-18,PA,Óbidos/PA,1505106,0,166,25,8717,316.353,16612.353,0.019
430967,311,2023-03-18,SP,Óleo/SP,3533809,0,6,0,162,245.198,6620.351,0.037


- Dados censo

In [ ]:
dados_censo = pd.read_csv('dados_municipios_2010.csv', sep = ';', decimal = ',', encoding = 'latin1')

## Ajustar dados

In [ ]:
max_date = max(dados_brasil['date'])

In [ ]:
dados_brasil = dados_brasil[(dados_brasil['date'] == max_date) & \
                            (dados_brasil['state'] != 'TOTAL')]

In [ ]:
cols = ['ibgeID', 'date', 'state', 'city', 'totalCases', 'deaths']

In [ ]:
dados_brasil = dados_brasil[cols].reset_index(drop=True)

- Fórmula do índice de Gini

In [ ]:
def gini(array):
    """Calculate the Gini coefficient of a numpy array."""
    # based on bottom eq:
    # http://www.statsdirect.com/help/generatedimages/equations/equation154.svg
    # from:
    # http://www.statsdirect.com/help/default.htm#nonparametric_methods/gini.htm
    # All values are treated equally, arrays must be 1d:
    array = array.flatten()
    if np.amin(array) < 0:
        # Values cannot be negative:
        array -= np.amin(array)
    # Values cannot be 0:
    array += 0.0000001
    # Values must be sorted:
    array = np.sort(array)
    # Index per array element:
    index = np.arange(1,array.shape[0]+1)
    # Number of array elements:
    n = array.shape[0]
    # Gini coefficient:
    return ((np.sum((2 * index - n  - 1) * array)) / (n * np.sum(array)))

## Aqui começa o trabalho

1 – Quantas cidades existem em cada dataframe? (dados_municipios_gerais e
dados_municipios_covid)?

In [ ]:
# Organizar em ordem alfabética por Município

dados_censo.sort_values("Municipio", inplace=True)

In [ ]:
# retirar duplicatas

dados_censo.drop_duplicates(subset="Municipio",
                     keep=False, inplace=True)

In [ ]:
dados_municipios_gerais = dados_censo
dados_municipios_gerais

,ibgeID,Municipio,ESPVIDA,E_ANOSESTUDO,T_ANALF18M,RDPC,IDHM
5318,5200050,ABADIA DE GOIÁS,74.82,7.94,8.36,574.96,0.708
5319,5200100,ABADIÂNIA,75.46,9.31,11.67,519.87,0.689
151,1500107,ABAETETUBA,72.89,8.99,14.97,293.01,0.628
890,2300101,ABAIARA,69.87,9.70,31.89,229.74,0.628
1827,2900207,ABARÉ,71.55,7.33,29.03,220.16,0.575
...,...,...,...,...,...,...,...
2252,3101003,Águas Vermelhas,73.46,7.72,30.09,306.97,0.601
3924,4101150,ÂNGULO,74.62,9.54,12.11,641.37,0.721
1831,2900504,ÉRICO CARDOSO,70.47,8.68,32.52,258.28,0.584
231,1505106,ÓBIDOS,70.35,8.89,13.73,255.03,0.594


In [ ]:
# total de Municipios em dados_municipios_gerais

len(dados_municipios_gerais['ibgeID'].unique())

5127

In [ ]:
# Organizar os dados por city

dados_brasil.sort_values("city", inplace=True)

In [ ]:
# retirar os valores duplicados

dados_brasil.drop_duplicates(subset="city",
                     keep=False, inplace=True)

In [ ]:
dados_municipios_covid = dados_brasil
dados_municipios_covid

,ibgeID,date,state,city,totalCases,deaths
0,5200050,2023-03-18,GO,Abadia de Goiás/GO,4074,50
1,3100104,2023-03-18,MG,Abadia dos Dourados/MG,1943,21
2,5200100,2023-03-18,GO,Abadiânia/GO,2218,55
3,1500107,2023-03-18,PA,Abaetetuba/PA,12009,244
4,3100203,2023-03-18,MG,Abaeté/MG,4279,51
...,...,...,...,...,...,...
5591,4301552,2023-03-18,RS,Áurea/RS,665,7
5592,4101150,2023-03-18,PR,Ângulo/PR,967,11
5593,2900504,2023-03-18,BA,Érico Cardoso/BA,2467,53
5594,1505106,2023-03-18,PA,Óbidos/PA,8717,166


In [ ]:
# Total de municipios em dados_municipios_covid

len(dados_municipios_covid['city'].unique())

5596

2) – Como realizar o join desses dataframes? (realizar o join!! Dica, usar a função pd.merge).
Indicar quais cidades não houve o join e retirar das análises.

In [ ]:
# Alterar o nome da coluna 'city' para Municipio

dados_municipios_covid_rename = dados_municipios_covid.rename(columns = {'city':'Municipio'})

In [ ]:
dados_municipios_covid_rename

,ibgeID,date,state,Municipio,totalCases,deaths
0,5200050,2023-03-18,GO,Abadia de Goiás/GO,4074,50
1,3100104,2023-03-18,MG,Abadia dos Dourados/MG,1943,21
2,5200100,2023-03-18,GO,Abadiânia/GO,2218,55
3,1500107,2023-03-18,PA,Abaetetuba/PA,12009,244
4,3100203,2023-03-18,MG,Abaeté/MG,4279,51
...,...,...,...,...,...,...
5591,4301552,2023-03-18,RS,Áurea/RS,665,7
5592,4101150,2023-03-18,PR,Ângulo/PR,967,11
5593,2900504,2023-03-18,BA,Érico Cardoso/BA,2467,53
5594,1505106,2023-03-18,PA,Óbidos/PA,8717,166


In [ ]:
dados_municipios_gerais

,ibgeID,Municipio,ESPVIDA,E_ANOSESTUDO,T_ANALF18M,RDPC,IDHM
5318,5200050,ABADIA DE GOIÁS,74.82,7.94,8.36,574.96,0.708
5319,5200100,ABADIÂNIA,75.46,9.31,11.67,519.87,0.689
151,1500107,ABAETETUBA,72.89,8.99,14.97,293.01,0.628
890,2300101,ABAIARA,69.87,9.70,31.89,229.74,0.628
1827,2900207,ABARÉ,71.55,7.33,29.03,220.16,0.575
...,...,...,...,...,...,...,...
2252,3101003,Águas Vermelhas,73.46,7.72,30.09,306.97,0.601
3924,4101150,ÂNGULO,74.62,9.54,12.11,641.37,0.721
1831,2900504,ÉRICO CARDOSO,70.47,8.68,32.52,258.28,0.584
231,1505106,ÓBIDOS,70.35,8.89,13.73,255.03,0.594


In [ ]:
# Fazer o join dos dados utilizando a coluna 'ibgeID' pois os dados dela não se repetem

dados_merge = pd.merge(dados_municipios_gerais, dados_municipios_covid_rename, on=['ibgeID'], how = 'left')

In [ ]:
dados_merge.dtypes

ibgeID            int64
Municipio_x      object
ESPVIDA         float64
E_ANOSESTUDO    float64
T_ANALF18M      float64
RDPC            float64
IDHM            float64
date             object
state            object
Municipio_y      object
totalCases        int64
deaths            int64
dtype: object

In [ ]:
# Tabela única com os dados após o merge

dados_merge

,ibgeID,Municipio_x,ESPVIDA,E_ANOSESTUDO,T_ANALF18M,RDPC,IDHM,date,state,Municipio_y,totalCases,deaths
0,5200050,ABADIA DE GOIÁS,74.82,7.94,8.36,574.96,0.708,2023-03-18,GO,Abadia de Goiás/GO,4074,50
1,5200100,ABADIÂNIA,75.46,9.31,11.67,519.87,0.689,2023-03-18,GO,Abadiânia/GO,2218,55
2,1500107,ABAETETUBA,72.89,8.99,14.97,293.01,0.628,2023-03-18,PA,Abaetetuba/PA,12009,244
3,2300101,ABAIARA,69.87,9.70,31.89,229.74,0.628,2023-03-18,CE,Abaiara/CE,1362,20
4,2900207,ABARÉ,71.55,7.33,29.03,220.16,0.575,2023-03-18,BA,Abaré/BA,3008,18
...,...,...,...,...,...,...,...,...,...,...,...,...
5122,3101003,Águas Vermelhas,73.46,7.72,30.09,306.97,0.601,2023-03-18,MG,Águas Vermelhas/MG,573,13
5123,4101150,ÂNGULO,74.62,9.54,12.11,641.37,0.721,2023-03-18,PR,Ângulo/PR,967,11
5124,2900504,ÉRICO CARDOSO,70.47,8.68,32.52,258.28,0.584,2023-03-18,BA,Érico Cardoso/BA,2467,53
5125,1505106,ÓBIDOS,70.35,8.89,13.73,255.03,0.594,2023-03-18,PA,Óbidos/PA,8717,166


In [ ]:
# Para descobrir os municípios ausentes no Join, devemos achar a diferença entra o número de municipios das duas tabelas usando o ibgeID como referência.
# Dessa forma a tabela gerada abaixo indica os 469 municipios que não foram incluídos na análise, uma vez que, há municipios com o nome idêncito, porém, de UF diferente.

dif = dados_municipios_covid_rename[~dados_municipios_covid_rename['ibgeID'].isin(dados_merge['ibgeID'])]
dif

,ibgeID,date,state,Municipio,totalCases,deaths
58,2500502,2023-03-18,PB,Alagoinha/PB,2557,26
59,2600609,2023-03-18,PE,Alagoinha/PE,3671,31
110,1400050,2023-03-18,RR,Alto Alegre/RR,2680,39
111,4300554,2023-03-18,RS,Alto Alegre/RS,457,3
112,3501103,2023-03-18,SP,Alto Alegre/SP,424,29
...,...,...,...,...,...,...
5534,2517100,2023-03-18,PB,Várzea/PB,684,1
5535,2414704,2023-03-18,RN,Várzea/RN,576,13
5562,2700102,2023-03-18,AL,Água Branca/AL,1115,31
5563,2500106,2023-03-18,PB,Água Branca/PB,2201,16


In [ ]:
municipios_fora_analise = dados_municipios_covid_rename['Municipio'].tolist()

In [ ]:
# Seguem uma resposta alternativa com os municipios fora da análise em forma de lista
print(municipios_fora_analise)

['Abadia de Goiás/GO', 'Abadia dos Dourados/MG', 'Abadiânia/GO', 'Abaetetuba/PA', 'Abaeté/MG', 'Abaiara/CE', 'Abaré/BA', 'Abatiá/PR', 'Abaíra/BA', 'Abdon Batista/SC', 'Abel Figueiredo/PA', 'Abelardo Luz/SC', 'Abre Campo/MG', 'Abreu e Lima/PE', 'Abreulândia/TO', 'Acaiaca/MG', 'Acajutiba/BA', 'Acarape/CE', 'Acaraú/CE', 'Acari/RN', 'Acará/PA', 'Acauã/PI', 'Aceguá/RS', 'Acopiara/CE', 'Acorizal/MT', 'Acrelândia/AC', 'Acreúna/GO', 'Adamantina/SP', 'Adelândia/GO', 'Adolfo/SP', 'Adrianópolis/PR', 'Adustina/BA', 'Afogados da Ingazeira/PE', 'Afonso Bezerra/RN', 'Afonso Cláudio/ES', 'Afonso Cunha/MA', 'Afrânio/PE', 'Afuá/PA', 'Agrestina/PE', 'Agricolândia/PI', 'Agrolândia/SC', 'Agronômica/SC', 'Aguanil/MG', 'Aguaí/SP', 'Agudo/RS', 'Agudos do Sul/PR', 'Agudos/SP', 'Aguiar/PB', 'Aguiarnópolis/TO', 'Aimorés/MG', 'Aiquara/BA', 'Aiuaba/CE', 'Aiuruoca/MG', 'Ajuricaba/RS', 'Alagoa Grande/PB', 'Alagoa Nova/PB', 'Alagoa/MG', 'Alagoinha do Piauí/PI', 'Alagoinha/PB', 'Alagoinha/PE', 'Alagoinhas/BA', 'Alamba

3) – No dataset após o join, obter o índice de Gini por UF (state). Qual UF chamou atenção? Por
que isto ocorreu?

In [ ]:
def gini(array):
    """Calculate the Gini coefficient of a numpy array."""
    # based on bottom eq:
    # http://www.statsdirect.com/help/generatedimages/equations/equation154.svg
    # from:
    # http://www.statsdirect.com/help/default.htm#nonparametric_methods/gini.htm
    # All values are treated equally, arrays must be 1d:
    array = array.flatten()
    if np.amin(array) < 0:
        # Values cannot be negative:
        array -= np.amin(array)
    # Values cannot be 0:
    array += 0.0000001
    # Values must be sorted:
    array = np.sort(array)
    # Index per array element:
    index = np.arange(1,array.shape[0]+1)
    # Number of array elements:
    n = array.shape[0]
    # Gini coefficient:
    return ((np.sum((2 * index - n  - 1) * array)) / (n * np.sum(array)))

In [ ]:
# selecionar apena as colunas relevantes para o cálculo

dados_merge_final = dados_merge[['ibgeID', 'RDPC', 'state']]

In [ ]:
dados_merge_final

,ibgeID,RDPC,state
0,5200050,574.96,GO
1,5200100,519.87,GO
2,1500107,293.01,PA
3,2300101,229.74,CE
4,2900207,220.16,BA
...,...,...,...
5122,3101003,306.97,MG
5123,4101150,641.37,PR
5124,2900504,258.28,BA
5125,1505106,255.03,PA


In [ ]:
# função abaixo para aplicar a função gini em mais de uma coluna

gini_UF = dados_merge_final.groupby('state')['RDPC'].apply(lambda x: gini(x.to_numpy()))

In [ ]:
# índice gini por UF

print(gini_UF)

state
AC    0.155132
AL    0.145568
AM    0.179084
AP    0.181761
BA    0.165484
CE    0.134921
DF    0.000000
ES    0.155466
GO    0.137378
MA    0.190558
MG    0.192143
MS    0.132975
MT    0.154945
PA    0.202667
PB    0.145239
PE    0.174080
PI    0.147741
PR    0.135024
RJ    0.152479
RN    0.143350
RO    0.130617
RR    0.183789
RS    0.155359
SC    0.136167
SE    0.142328
SP    0.141167
TO    0.165443
Name: RDPC, dtype: float64


In [ ]:
# Note que a UF do Distrito Federal (DF) apresentou valor 0 o índice Gini, isso ocorreu pois a amostragem dessa UF é de apenas 1 município, que seria Brasilia no caso, então,
# como só há um município dentro de DF, o índice gini não teve como comparar com outros dados e assim calcular corretamente

# Vale dizer que o índice gini quanto mais próximo de 0, menor é a desigualdade, porém, o fato de DF ter apresentado 0 não significa que não há desigualdade lá, mas que a amostragem e a metodologia
# aplicada não condiz com essa realidade específica

4) Obtenha o sumário das colunas por UF (state), similar a figura abaixo (apresentar o código):

In [ ]:
# organizando a tabela para análise com as colunas desejadas
dados_sumario = dados_merge[['state', 'totalCases', 'deaths', 'ESPVIDA', 'E_ANOSESTUDO', 'T_ANALF18M', 'IDHM', 'RDPC']]

In [ ]:
dados_sumario

,state,totalCases,deaths,ESPVIDA,E_ANOSESTUDO,T_ANALF18M,IDHM,RDPC
0,GO,4074,50,74.82,7.94,8.36,0.708,574.96
1,GO,2218,55,75.46,9.31,11.67,0.689,519.87
2,PA,12009,244,72.89,8.99,14.97,0.628,293.01
3,CE,1362,20,69.87,9.70,31.89,0.628,229.74
4,BA,3008,18,71.55,7.33,29.03,0.575,220.16
...,...,...,...,...,...,...,...,...
5122,MG,573,13,73.46,7.72,30.09,0.601,306.97
5123,PR,967,11,74.62,9.54,12.11,0.721,641.37
5124,BA,2467,53,70.47,8.68,32.52,0.584,258.28
5125,PA,8717,166,70.35,8.89,13.73,0.594,255.03


In [ ]:
#criando o dataset com a função abaixo com os indicadores desejados em suas respectivas colunas

dados_sumario.groupby('state') \
             .agg(casos_totais = pd.NamedAgg('totalCases','sum'),
                  mortes_totais = pd.NamedAgg('deaths','sum'),
                  media_esp_vida = pd.NamedAgg('ESPVIDA','mean'),
                  media_anos_estudo = pd.NamedAgg('E_ANOSESTUDO','mean'),
                  media_t_anal = pd.NamedAgg('T_ANALF18M', 'mean'),
                  mediana_idhm = pd.NamedAgg('IDHM', 'median'),
                  mediana_rdpc = pd.NamedAgg('RDPC','median'),
                  dp_rdpc = pd.NamedAgg('RDPC', 'std'))\
              .reset_index()

,state,casos_totais,mortes_totais,media_esp_vida,media_anos_estudo,media_t_anal,mediana_idhm,mediana_rdpc,dp_rdpc
0,AC,58046,647,70.869500,7.585000,27.214000,0.5820,305.730,84.956732
1,AL,311820,6769,69.315000,8.927209,35.499419,0.5605,233.005,82.799485
2,AM,611834,14109,70.885345,7.447759,21.076207,0.5630,227.105,98.919979
3,AP,151208,1918,71.986667,8.977333,14.220667,0.6400,397.980,136.210025
4,BA,1658551,29496,70.611963,8.442016,25.935942,0.5880,271.560,105.365780
5,CE,1331616,26502,70.633533,9.680120,29.796228,0.6110,250.790,78.148445
6,DF,898072,11845,77.350000,9.870000,3.660000,0.8240,1715.110,NaN
7,ES,1270686,14209,74.541486,9.222973,13.303243,0.6835,540.835,207.574809
8,GO,1820165,26778,74.473723,9.783290,13.726450,0.6970,569.960,147.057719
9,MA,449658,10294,69.393401,9.045178,29.480914,0.5700,220.910,91.897980


5 – Por meio das análises na tabela obtida na questão 4, pode-se presumir algum resultado interessante? Apresente sua opinião.

In [ ]:
# Uma observação interessante é o fato de que o Distrito Federal (DF) possuir os maior índice de expectativa de vida e de mediana de renda per capita, uma vez que,
# por ser o centro Político do Brasil, boa parte dos habitantes possuem cargos públicos com bons salários, dessa forma, maiores salários fazem com que a expectativa de vida tende a aumentar

# Vale destacar o quão desigual o Brasil é regionalmente, onde a região Sul e Sudeste demonstram altos índices de média de anos estudados, expectativa de vida e mediana IDH
# e por outro lado mostra os Estados do Norte e Nordeste em sua maioria com índices piores, destacando negativamente o Estado do Alagoas com o menor índice de mediana IDH do país.

# Outra análise interessante, é que os casos de covid e mortes não necessariamente seguem os indicadores sociais de desenvolvimento e renda, por exemplo,
# não necessariamente estados com grandes números de mortos possuem indicadores como mediana IDH ou mediana renda per capita baixos, pois, os casos e mortes de covid estão mais atribuidos a
# números demográficos do que necessariamente indicadores sociais

In [ ]:
# INTEGRANTES
# Cesar Oliveira Goes - RM: 98119
# Fiama dos Santos Trajano - RM: 97885
# Gabriel Silva de Negreiros Leal da Rocha - RM: 550759
# Karina Maciel Palmeira - RM: 551770